In [ ]:

import time
from Arm_Lib import Arm_Device

# Create arm object (make sure DOFBOT USB is connected)
arm = Arm_Device()
time.sleep(0.2)

print("Arm connected.")


In [ ]:

# - Helper functions -

def open_gripper():
    # Adjust the angle if your gripper does not fully open
    arm.Arm_serial_servo_write(6, 60, 400)
    time.sleep(0.4)

def close_gripper():
    # Adjust the angle if your gripper does not fully close
    arm.Arm_serial_servo_write(6, 135, 400)
    time.sleep(0.4)


def move_arm(angles, duration=800):
    """angles: [s1, s2, s3, s4, s5]"""
    for i in range(5):
        sid = i + 1
        angle = angles[i]
        t = int(duration * 1.2) if sid == 5 else duration
        arm.Arm_serial_servo_write(sid, angle, t)
        time.sleep(0.01)
    time.sleep(duration / 1000.0)


def go_up():
    # Simple "up" posture – tune if needed
    arm.Arm_serial_servo_write(2, 90, 1000)
    arm.Arm_serial_servo_write(3, 90, 1000)
    arm.Arm_serial_servo_write(4, 90, 1000)
    time.sleep(0.8)


def go_home():
    move_arm(home_pos, duration=900)


print("Helper functions ready.")


In [ ]:

# rest
home_pos = [90, 130, 0, 0, 90]

# Above pickup point
pick_above = [90, 80, 50, 50, 270]

# Exact pickup point
pick_pos = [90, 53, 33, 36, 270]

# Basket positions (example layout)
basket_yellow = [65, 22, 64, 56, 270]
basket_red    = [117, 19, 66, 56, 270]
basket_green  = [136, 66, 20, 29, 270]
basket_blue   = [44, 66, 20, 28, 270]

print("Positions loaded.")


In [ ]:

# - Class → Basket mapping -


fruit_to_basket = {
    "banana": basket_yellow,
    "orange": basket_red,
    "apple":  basket_green,
    "lemon":  basket_blue,
 
}

def basket_for_class(name):
    name = name.strip().lower()
    if name in fruit_to_basket:
        return fruit_to_basket[name]
    print(f"No basket defined for '{name}'.")
    return None

print("Mapping ready.")


In [ ]:

# - Pick and place routine -

def pick_and_place(target_pos, label=""):
    if target_pos is None:
        return

    print(f"Sorting: {label}")

    # Make sure that gripper is open and arm is at home
    open_gripper()
    go_home()

    # Move above pickup
    move_arm(pick_above, duration=1000)

    # Move down to pickup
    move_arm(pick_pos, duration=1000)

    # Grab fruit
    close_gripper()

    # Go back up
    move_arm(pick_above, duration=1000)

    # Move to basket
    move_arm(target_pos, duration=1000)

    # Drop fruit
    open_gripper()

    # Lift a bit and go home
    go_up()
    go_home()

    print("Done. Ready for next one.")


In [ ]:


#type the class name.

def get_detected_class():
    text = input("Fruit class (apple/banana/orange/lemon or q to quit): ").strip().lower()
    return text

print("Stub detection function ready.")


In [ ]:

# - Main loop -

try:
    print("Starting manual sorting loop...")

    while True:
        cls = get_detected_class()
        if cls in ["q", "quit", "exit"]:
            print("Exiting loop.")
            break

        if not cls:
            print("Empty input, try again.")
            continue

        target = basket_for_class(cls)
        if target is None:
            # Unknown class name
            continue

        pick_and_place(target_pos=target, label=cls)

except KeyboardInterrupt:
    print("Interrupted by user.")

print("Loop ended.")


In [ ]:

# Test added: clean up the arm object when completed

try:
    del arm
    print("Arm object deleted.")
except NameError:
    print("Arm was already deleted or not defined.")
